In [1]:
import glacierml as gl
import pandas as pd
import numpy as np
from tqdm import tqdm
RS = range(0,25,1)


In [2]:
parameterization = '4'
if parameterization == '1':
    threshold = 'No Threshold'
if parameterization == '2':
    threshold = '0.25'
if parameterization == '3':
    threshold = '0.50'
if parameterization == '4':
    threshold = '0.75'
dataset = gl.parameterize_data(parameterization)
dataset

,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Thickness
0,63.469173,-146.528168,12.0,722,1841,3081,234.583,30,41911,235.0
1,63.281000,-145.427000,14.0,1162,1858,2438,17.567,172,8639,147.0
2,58.380000,-134.349000,10.0,676,1123,1494,9.528,327,6332,66.0
8,61.200000,-140.302000,14.0,2145,2509,2894,5.859,341,3374,66.0
9,51.177000,-116.330000,15.3,2390,2526,2853,0.872,69,1181,70.0
...,...,...,...,...,...,...,...,...,...,...
489,-33.029600,-70.098400,29.0,2929,5121,5894,8.376,358,8560,59.0
490,-30.145400,-69.927600,29.3,4664,5187,5525,1.189,148,1596,28.0
491,-28.538700,-69.731800,16.0,4784,5234,5597,3.140,190,3349,39.0
492,-28.381400,-69.609100,11.6,5068,5406,5753,5.209,79,3782,67.0


In [3]:
# load deviations table 
# rootdir = 'zults/'
rootdir = '/data/fast1/glacierml/data/zults/'
predictions = pd.DataFrame()
statistics = pd.DataFrame()
file_reader = pd.read_csv(rootdir + 'model_statistics_' + parameterization + '.csv')
statistics = pd.concat([statistics, file_reader], ignore_index = True)

statistics = statistics.drop('Unnamed: 0', axis = 1)
statistics['total parameters'] = statistics['total parameters'].astype(int)
statistics['trained parameters'] = statistics['trained parameters'].astype(int)
statistics['total inputs'] = statistics['total inputs'].astype(int)
statistics['test - train'] = (
    abs(statistics['test mae avg'] - statistics['train mae avg']))
statistics['paramater ratio'] = statistics['trained parameters'] / statistics['total inputs']
statistics

,layer architecture,total parameters,trained parameters,total inputs,test mae avg,train mae avg,test mae std dev,train mae std dev,architecture weight 1,architecture weight 2,test - train,paramater ratio
0,3-2,60,41,3069,17.888150,18.320922,4.794239,3.915898,145.110860,0.006891,0.432772,0.013359
1,4-2,72,53,3069,19.495260,19.487580,5.440397,4.502279,133.148514,0.007510,0.007680,0.017269
2,5-2,84,65,3069,16.948900,17.207373,4.296981,3.105293,153.152412,0.006529,0.258473,0.021180
3,6-2,96,77,3069,17.472450,17.441128,5.753468,3.278046,148.563301,0.006731,0.031322,0.025090
4,7-2,108,89,3069,18.054400,18.706029,4.467628,4.463059,143.774640,0.006955,0.651629,0.029000
...,...,...,...,...,...,...,...,...,...,...,...,...
156,16-15,450,431,3069,15.679167,16.010148,3.222594,0.768343,165.555028,0.006040,0.330981,0.140437
157,17-15,475,456,3069,15.743689,16.066999,3.108120,0.780996,164.876527,0.006065,0.323310,0.148583
158,18-15,500,481,3069,15.769674,16.090924,3.054389,0.852239,164.604852,0.006075,0.321250,0.156729
159,19-15,525,506,3069,15.665721,16.042760,3.150946,0.752430,165.697122,0.006035,0.377039,0.164875


In [4]:
df = pd.DataFrame()
for i in tqdm(range(0,161,1)):
    
    selection = i
    arch = statistics['layer architecture'].loc[selection]

    rootdir_1 = 'saved_results/' + parameterization + '/' + arch + '/'
    dnn_history = {}


    dft = pd.DataFrame()
    for n, rs in enumerate(RS): 
        dftt = pd.DataFrame()
        history_name = (

            str(rs)
        )

        model_name = (

            str(rs)
        )

        dnn_history[model_name] = pd.read_csv(rootdir_1 + model_name)
        
        MAE = pd.Series(dnn_history[model_name]['val_loss'].iloc[-1], name = 'mae')
        architecture = pd.Series(arch, name = 'arch')
        model_name = pd.Series(model_name, name = 'Random State')
        
        dftt = pd.concat([
            dftt, MAE, architecture, model_name
        ], axis = 1)
                
        dft = pd.concat([
            dft,
            dftt
        ])
        
        
    dft = dft.reset_index()
    dft = dft.drop('index', axis = 1)
    dft['mean'] = dft['mae'].mean()
    dft['var'] = dft['mae'].var()
    df = pd.concat([df, dft])
        

df

100%|██████████| 161/161 [00:11<00:00, 14.57it/s]


,mae,arch,Random State,mean,var
0,16.385727,3-2,0,19.148526,23.606710
1,16.956924,3-2,1,19.148526,23.606710
2,17.843706,3-2,2,19.148526,23.606710
3,16.616665,3-2,3,19.148526,23.606710
4,17.377512,3-2,4,19.148526,23.606710
...,...,...,...,...,...
20,16.212910,20-15,20,16.480931,0.149075
21,16.177799,20-15,21,16.480931,0.149075
22,16.698284,20-15,22,16.480931,0.149075
23,16.774265,20-15,23,16.480931,0.149075


In [5]:
stats = pd.DataFrame()
for i in df['arch'].unique():
    dft = df[df['arch'] == i]
#     print(dft[['mean', 'var']].iloc[-1])
    dftt = dft[['arch','mean', 'var']].iloc[-1]
    stats = pd.concat([stats, dftt], axis = 1)
    
#     break
stats = stats.T
stats

,arch,mean,var
24,3-2,19.148526,23.60671
24,4-2,20.951514,33.232499
24,5-2,18.140492,15.649445
24,6-2,18.531582,20.140801
24,7-2,19.908215,29.583556
...,...,...,...
24,16-15,16.208994,0.075997
24,17-15,16.398673,0.121314
24,18-15,16.318113,0.164099
24,19-15,16.305696,0.27803


# Implementing Graybill-Deal estimator for common mean and variance

$\hat{\mu}_{GD} = \sum_{i}^{N_x} \frac{n_{i}}{S_{i}^{2}} \bar{X}_{i} \big{/} \sum_{i}^{N_x} \frac{n_{i}}{S_{i}^{2}}$

Var$\left(\hat{\mu}_{GD}\right) = E\big{[} \left( \sum_{i}^{N_x} \frac{n_{i}\sigma_{i}^{2}}{S_{i}^{4}} \right) \big{/} \left( \sum_{i}^{N_x}\frac{n_{i}}{S_{i}^{2}} \right)^{2}\big{]}$ 

In [6]:
bar_X = stats['mean']
S_sq = stats['var']
n = len(stats)

hat_mu = np.round(sum(
    ( n / S_sq ) * bar_X  
) / sum(  
    n / S_sq
), 3)
print(
    f'Graybill-Deal Common Mean Estimation of model Mean Absolute Error: {hat_mu} m'
)

Graybill-Deal Common Mean Estimation of model Mean Absolute Error: 16.321 m


In [7]:
var_mu_GD = ((
    sum(
        (n * S_sq) / S_sq**2
    ) / sum(
        n / S_sq
    )**2
))
var_mu_GD
print(
    f'Variance of Graybill-Deal Common Mean Estimation of model Mean Absolute Error: '
    f'{var_mu_GD} m'
)

Variance of Graybill-Deal Common Mean Estimation of model Mean Absolute Error: 9.015530973746864e-06 m


# MAE uncertainty $\tilde{\sigma}^{2}_{\mu} = \hat{\mu}_{GD} = 16.321 m$